In [1]:
import numpy as np
import pandas as pd

In [2]:
working_dir = '~/Project/pred-future-sales'
sales_train = pd.read_csv(working_dir + '/data/interim/sales_train.csv')
train_test = pd.read_csv(working_dir + '/data/interim/train_test.csv')

In [4]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,2013-02-01,0,59,22154,999.00,1.0,999.00
1,2013-03-01,0,25,2552,899.00,1.0,899.00
2,2013-05-01,0,25,2552,899.00,-1.0,-899.00
3,2013-06-01,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00


In [5]:
train_test.head()

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code
0,0,0,19,0.0,29,40,11,6
1,0,0,27,0.0,29,19,5,12
2,0,0,28,0.0,29,30,8,57
3,0,0,29,0.0,29,23,5,18
4,0,0,32,6.0,29,40,11,6


In [6]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num', 'shop_id', 'item_id', col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', 'shop_id', 'item_id', col + '_lag_' + str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', 'shop_id', 'item_id'], how='left')
    return df

In [9]:
train_test = lag_feature(train_test, [1, 2, 3, 6, 12], 'item_cnt_month')

In [ ]:
group = train_test.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = ['item_cnt_month_avg']
group.reset_index(inplace=True)
train_test = pd.merge(train_test, group)